In [1]:
import torch
from numba import cuda
torch.cuda.empty_cache()
cuda.select_device(0)
cuda.close()

import os
os.chdir('/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/src/vendor_run/mellotron')

from hparams import create_hparams
from model import Tacotron2
from text import cmudict, text_to_sequence
from train import warm_start_model
from loss_function import Tacotron2Loss
import waveglow.glow
from waveglow.denoiser import Denoiser
import waveglow
import sys
import IPython.display as ipd
from train import prepare_dataloaders
import time 
sys.path.append('waveglow/')
def to_arpabet(text):
    from g2p_en import G2p
    g2p = G2p()
    return " ".join(
        [
            f"{{ {s.strip()} }}" if s.strip() not in ",." else s.strip()
            for s in " ".join(g2p(text)).split("  ")
        ]
    )

def load_model(hparams):
    model = Tacotron2(hparams).cuda()
    if hparams.fp16_run:
        model.decoder.attention_layer.score_mask_value = finfo('float16').min

    return model




In [2]:

waveglow_path = '../../vendor_tools/waveglow/waveglow_256channels_universal_v4.pt'
waveglow_2 = torch.load(waveglow_path)['model'].cuda().eval()
denoiser = Denoiser(waveglow_2).cuda().eval()

/opt/conda/lib/python3.7/site-packages/torch/serialization.py:671: SourceChangeWarning: source code of class 'torch.nn.modules.conv.ConvTranspose1d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/opt/conda/lib/python3.7/site-packages/torch/serialization.py:671: SourceChangeWarning: source code of class 'torch.nn.modules.container.ModuleList' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/opt/conda/lib/python3.7/site-packages/torch/serialization.py:671: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv1d' has changed. you can retrieve the original source code by accessing the object's source attribute or

In [3]:
hparams = create_hparams(None)
hparams.n_speakers = 1
hparams.training_files = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/filelists/lj_train.txt'
hparams.validation_files = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/filelists/lj_val.txt'
hparams.batch_size = 16
hparams.checkpoint_path = '/mnt/disks/uberduck-experiments-v0/models/mellotron/vendor/mellotron_ljs.pt'
hparams.warm_start = True
learning_rate = 1e-3#
torch.manual_seed(hparams.seed)
model = load_model(hparams)
model = warm_start_model(hparams.checkpoint_path, model, hparams.ignore_layers)
arpabet_dict = cmudict.CMUDict('data/cmu_dictionary')


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Warm starting model from checkpoint '/mnt/disks/uberduck-experiments-v0/models/mellotron/vendor/mellotron_ljs.pt'


In [4]:
text = "its like this and like that and like this and uh"
text = to_arpabet(text)
text_encoded = torch.LongTensor(text_to_sequence(text, hparams.text_cleaners, arpabet_dict))[None, :].cuda()    
speaker_ids = torch.tensor([0], device='cuda:0').to(torch.int64)

xnew =(text_encoded, 0,speaker_ids,torch.zeros([1, 1, 481], device =  'cuda:0'))
with torch.no_grad():
    model.zero_grad()
    mel_outputs, mel_outputs_postnet, gate_outputs, rhythm = model.inference(xnew)
    audio = denoiser(waveglow_2.infer(mel_outputs_postnet, sigma=0.8), 0.01)[:, 0]
ipd.Audio(audio[0].data.cpu().numpy(), rate=hparams.sampling_rate)

In [4]:
criterion = Tacotron2Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay=hparams.weight_decay)
optimizer.eps = 1e-16

In [11]:
import tensorflow as tf
#import tensorflow.keras as keras
#from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.3
set_session(tf.Session(config=config))

AttributeError: module 'tensorflow' has no attribute 'set_session'

In [16]:
from tensorflow import keras 

In [17]:
from keras.backend.tensorflow_backend import set_session

ImportError: cannot import name 'get_config' from 'tensorflow.python.eager.context' (/opt/conda/lib/python3.7/site-packages/tensorflow_core/python/eager/context.py)

In [7]:
import sys
!{sys.executable} -m pip install keras

  Using cached keras-2.6.0-py2.py3-none-any.whl (1.3 MB)


In [5]:
train_loader, valset, collate_fn, train_sampler = prepare_dataloaders(hparams)
timestart = time.time()
for epoch in range(5):
    for i, batch in enumerate(train_loader):
        
        x, y = model.parse_batch(batch)
        model.zero_grad()
        y_pred = model(x)
        loss = criterion(y_pred, y)
        reduced_loss = loss.item()
        loss.backward()
        optimizer.step()
        timeend = time.time()
        print(epoch,i, timestart - timeend)
        timestart = timeend

here


RuntimeError: cuDNN error: CUDNN_STATUS_NOT_INITIALIZED

In [ ]:
xnew =(text_encoded, 0,speaker_ids,torch.zeros([1, 1, 481], device =  'cuda:0'))
with torch.no_grad():
    model.zero_grad()
    mel_outputs, mel_outputs_postnet, gate_outputs, rhythm = model.inference(xnew)
    audio = denoiser(waveglow_2.infer(mel_outputs_postnet, sigma=0.8), 0.01)[:, 0]

In [8]:
import torch
from numba import cuda
torch.cuda.empty_cache()
cuda.select_device(0)
cuda.close()

In [3]:
hparams = create_hparams(None)
hparams.n_speakers = 1
hparams.training_files = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/filelists/lj_train.txt'
hparams.validation_files = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/filelists/lj_val.txt'
hparams.batch_size = 16
hparams.checkpoint_path = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/outputs/ljmel/checkpoint_500'

hparams.warm_start = True
learning_rate = 1e-3#
torch.manual_seed(hparams.seed)
model = load_model(hparams)
model = warm_start_model(hparams.checkpoint_path, model, hparams.ignore_layers)
arpabet_dict = cmudict.CMUDict('data/cmu_dictionary')
text = "its like this and like that and like this and uh"
text = to_arpabet(text)
text_encoded = torch.LongTensor(text_to_sequence(text, hparams.text_cleaners, arpabet_dict))[None, :].cuda()    
speaker_ids = torch.tensor([0], device='cuda:0').to(torch.int64)

xnew =(text_encoded, 0,speaker_ids,torch.zeros([1, 1, 481], device =  'cuda:0'))
with torch.no_grad():
    model.zero_grad()
    mel_outputs, mel_outputs_postnet, gate_outputs, rhythm = model.inference(xnew)
    audio = denoiser(waveglow_2.infer(mel_outputs_postnet, sigma=0.8), 0.01)[:, 0]

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Warm starting model from checkpoint '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/outputs/ljmel/checkpoint_500'
Warning! Reached max decoder steps


In [4]:
ipd.Audio(audio[0].data.cpu().numpy(), rate=hparams.sampling_rate)